In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import pandas as pd
import io
import numpy as np
import time
from fuzzywuzzy import fuzz, process
from unidecode import unidecode
from tqdm import tqdm

In [2]:
positions = pd.read_csv("../data/playerpositions.csv")
player_names_positions = positions['Players'].tolist()
players_pic_link = pd.read_csv("../data/positions2.csv")
player_pic_link_names = players_pic_link['Players'].tolist()

In [34]:
nba_team_abbreviations = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BRK",
    "Buffalo Braves": "BUF",
    "Charlotte Hornets": "CHH",
    "Charlotte Bobcats": "CHA",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Kansas City Kings": "KCK",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Jersey Nets": "NJN",
    "New Orleans/Oklahoma City Hornets": "NOK",
    "New Orleans Pelicans": "NOP",
    "New Orleans Hornets": "NOH",
    "New Orleans Jazz": "NOJ",
    "New York Knicks": "NYK",
    "New York Nets": "NYN",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHO",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "San Diego Clippers": "SDC",
    "Seattle SuperSonics": "SEA",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Vancouver Grizzlies": "VAN",
    "Washington Bullets": "WSB",
    "Washington Wizards": "WAS"
    }


nba_team_names = {
    "ATL": "Atlanta Hawks",
    "BOS": "Boston Celtics",
    "BRK": "Brooklyn Nets",
    "BUF": "Buffalo Braves",
    "CHH": "Charlotte Hornets",
    "CHA": "Charlotte Bobcats",
    "CHI": "Chicago Bulls",
    "CLE": "Cleveland Cavaliers",
    "DAL": "Dallas Mavericks",
    "DEN": "Denver Nuggets",
    "DET": "Detroit Pistons",
    "GSW": "Golden State Warriors",
    "HOU": "Houston Rockets",
    "IND": "Indiana Pacers",
    "KCK": "Kansas City Kings",
    "LAC": "Los Angeles Clippers",
    "LAL": "Los Angeles Lakers",
    "MEM": "Memphis Grizzlies",
    "MIA": "Miami Heat",
    "MIL": "Milwaukee Bucks",
    "MIN": "Minnesota Timberwolves",
    "NJN": "New Jersey Nets",
    "NOK": "New Orleans/Oklahoma City Hornets",
    "NOP": "New Orleans Pelicans",
    "NOH": "New Orleans Hornets",
    "NOJ": "New Orleans Jazz",
    "NYK": "New York Knicks",
    "NYN": "New York Nets",
    "OKC": "Oklahoma City Thunder",
    "ORL": "Orlando Magic",
    "PHI": "Philadelphia 76ers",
    "PHO": "Phoenix Suns",
    "POR": "Portland Trail Blazers",
    "SAC": "Sacramento Kings",
    "SAS": "San Antonio Spurs",
    "SDC": "San Diego Clippers",
    "SEA": "Seattle SuperSonics",
    "TOR": "Toronto Raptors",
    "UTA": "Utah Jazz",
    "VAN": "Vancouver Grizzlies",
    "WSB": "Washington Bullets",
    "WAS": "Washington Wizards"
}

In [4]:
def get_all_links_all_games(big_df):
    length = len(big_df)
    link_list = []
    for i in range(length):
        df = big_df.iloc[[i]]
        link_list.append(get_boxscore_link(df))
    
    return link_list
        

In [5]:
def team_link(teamabv, year):
    return f'https://www.basketball-reference.com/teams/{teamabv}/{year}.html#all_roster'

In [6]:
def team_players(team_link):
    URL = team_link
    response = requests.get(URL)
    response.status_code
    soup = BeautifulSoup(response.text, features="html.parser")
    soup.prettify()
    tables = soup.find_all('table')
    table_html = str(tables[0])
    df = pd.read_html(io.StringIO(table_html))[0]
    players_list = df['Player'].tolist()
    players_list = list(map(lambda x: x.split('(')[0].strip(), players_list))
    return players_list

In [7]:
def player_link(name):
    
    firstname = name.split()[0]
    lastname = name.split()[1]
    firstname = "".join(char for char in firstname if char.isalpha()).lower()
    lastname = "".join(char for char in lastname if char.isalpha()).lower()
    lastinitial = lastname[0].lower()
    
    lastname5 = lastname[0:5]
    firstname2 = firstname[0:2]
    number = 1
    
    link = f'https://www.basketball-reference.com/players/{lastinitial}/{lastname5}{firstname2}0{number}.html'
    NotFound = True
    
    while NotFound:
        URL = link
        response = requests.get(URL)
        soup = BeautifulSoup(response.text, features="html.parser")
        h1_tag = soup.find('h1')
        player_name = h1_tag.text.strip()
        if player_name == name:
            print('Found it!')
            
            
            NotFound = False
        elif response.status_code != 200:
            print("Name Not Found....")
            NotFound = False
            positions = ['PG','SG','SF','PF','C']
        else:
            number += 1
            link = f'https://www.basketball-reference.com/players/{lastinitial}/{lastname5}{firstname2}0{number}.html'
            print(f'Didnt find it, trying again. This is {number} trying next.' )
        
    picturelink = f'https://www.basketball-reference.com/req/202106291/images/headshots/{lastname5}{firstname2}0{number}.jpg'
    linklist = []
    linklist.append(link)
    linklist.append(picturelink)
    return linklist

In [8]:
def get_positions_link(link, testing = False):
    response = requests.get(link)
    if response.status_code != 200:
        positions = ['PG','SG','SF','PF','C']
        return positions
    else:
        soup = BeautifulSoup(response.text, features="html.parser")
        inactive_section = soup.find_all('p')
        for i in range(10):
            try:
                if testing:
                    print(f'Trying {i}')
                
                positions = inactive_section[i]
                
                strong_tag = positions.find('strong', string=lambda t: t and "Position:" in t)
                position_text = strong_tag.next_sibling.strip()
                break
            except:
                pass
            
            
            
        positions = position_text.strip()
        only_positions = positions.split('\n')[0]
        
        if ',' in only_positions:
            only_positions = only_positions.split(',')
        else:
            if 'and' in only_positions:
                only_positions = only_positions.split('and')
            else:
                only_positions = [only_positions]
        if testing:
            print(f'This is only positions: {only_positions}')
            
            
        final_positions = []
        position_dict = {
            'Point Guard':"PG",
            'Shooting Guard' : "SG",
            'Small Forward' : "SF",
            'Power Forward' : "PF",
            'Center' : "C",
            'Forward' : "SF",
            'Guard' : 'PG'
        }
        for i in range(len(only_positions)):
            cleaned_string = only_positions[i].replace("and", "")
            if testing:
                print(f'This is cleaned_string: {cleaned_string}')
            cleaned_string = cleaned_string.strip()
            if testing:
                print(f'This is cleaned_string: {cleaned_string}')
            
            cleaned_string = position_dict[cleaned_string]
            
            final_positions.append(cleaned_string)
            
        
        return final_positions

In [9]:
def name_to_positions(name, testing = False):
    link = player_link(name)
    positions = get_positions_link(link[0], testing = testing)
    positionsandlink = []
    positionsandlink.append(positions)
    positionsandlink.append(link[1])
    return positionsandlink

In [10]:
def get_positions(team_abv,year):
    link = team_link(team_abv,year)
    list_players = team_players(link)
    
    
    
    names_list = []
    pos1_list = []
    pos2_list = []
    pos3_list = []
    pos4_list = []
    pos5_list = []
    picture_link = []
    for name in list_players:
        time.sleep(8)
        
        positionsAll = name_to_positions(name, False)
        positions = positionsAll[0]
        
        names_list.append(name)
        
        for i in range(5):
            if i == 0:
                try:
                    pos1_list.append(positions[i])
                except:
                    pos1_list.append(np.nan)
            if i == 1:
                try:
                    pos2_list.append(positions[i])
                except:
                    pos2_list.append(np.nan)
            if i == 2:
                try:
                    pos3_list.append(positions[i])
                except:
                    pos3_list.append(np.nan)
            if i == 3:
                try:
                    pos4_list.append(positions[i])
                except:
                    pos4_list.append(np.nan)
            if i == 4:
                try:
                    pos5_list.append(positions[i])
                except:
                    pos5_list.append(np.nan)
        
        picture_link.append(positionsAll[1])
        print(names_list)
        #print(pos1_list)
        #print(pos2_list)
        #print(pos3_list)
        #print(pos4_list)
        #print(pos5_list)
    
    
    df = pd.DataFrame(
        {
            'Players':names_list,
            'Pos1':pos1_list,
            'Pos2':pos2_list,
            'Pos3':pos3_list,
            'Pos4':pos4_list,
            'Pos5':pos5_list,
            'PictureLink':picture_link
            
        }
    )
    
    return df
        
    

In [11]:
def add_fantasy_averages(df):
    
    df = df.copy()  # Avoid modifying original dataframe
    df["_original_index"] = df.index  # Store original index
    
    df = df.sort_values(by=["Starters", "Date"])  # Sort by player and date

    # Calculate cumulative average per player
    df["AvgFantPoints"] = df.groupby("Starters")["FantasyPoints"].expanding().mean().reset_index(level=0, drop=True)

    df = df.sort_values(by="_original_index").drop(columns=["_original_index"])  # Restore original order

    return df

In [12]:
def add_fantasy_averages_new(df):
    
    df["AvgFantPoints"] = df.apply(lambda row: df[
    (df["Starters"] == row["Starters"]) &  # Same player
    (df["Date"] < row["Date"]) &  # Only past games
    (~df["DidNotPlay"])  # Only games where the player actually played
    ]["FantasyPoints"].mean(), axis=1)


    return df

In [13]:
def add_fantasy_averages_new_new(df):
    #df = df.sort_values(["Starters", "Date"])  # Ensure chronological order for each player

    avg_fantasy = []
    avg_fantasy_last10 = []
    avg_mp_time = []
    avg_mp_time_last10 = []
    avg_points = []
    avg_points_last10 = []
    avg_assists = []
    avg_assists_last10 = []
    avg_rebounds = []
    avg_rebounds_last10 = []
    avg_steals = []
    avg_steals_last10 = []
    avg_blocks = []
    avg_blocks_last10 = []
    avg_turnovers = []
    avg_turnovers_last10 = []
    avg_threepointers = []
    avg_threepointers_last10 = []
    
   

    # Convert MPTimeDelta to timedelta format
    df["MPTimeDelta"] = pd.to_timedelta(df["MPTimeDelta"])

    for idx, row in tqdm(df.iterrows()):
        past_games = df[
            (df["Starters"] == row["Starters"]) & 
            (df["Date"] < row["Date"]) & 
            (~df["DidNotPlay"])
        ].copy()  # ✅ Copy to avoid potential SettingWithCopyWarning

        # Convert MPTimeDelta to minutes as float
        past_games["MPMinutes"] = past_games["MPTimeDelta"].dt.total_seconds() / 60

        # Calculate averages
        avg_fantasy.append(past_games["FantasyPoints"].mean())
        avg_mp_time.append(past_games['MPMinutes'].mean())
        avg_points.append(past_games["PTS"].mean())
        avg_assists.append(past_games["AST"].mean())
        avg_rebounds.append(past_games["TRB"].mean())
        avg_steals.append(past_games["STL"].mean())
        avg_blocks.append(past_games["BLK"].mean())
        avg_turnovers.append(past_games["TOV"].mean())
        avg_threepointers.append(past_games["3P"].mean())

        last_10_games = past_games.tail(10).copy()  # ✅ Copy ensures "MPMinutes" exists

        avg_fantasy_last10.append(last_10_games["FantasyPoints"].mean())
        avg_mp_time_last10.append(last_10_games["MPMinutes"].mean() if not last_10_games.empty else None)
        avg_points_last10.append(last_10_games["PTS"].mean())
        avg_assists_last10.append(last_10_games["AST"].mean())
        avg_rebounds_last10.append(last_10_games["TRB"].mean())
        avg_steals_last10.append(last_10_games["STL"].mean())
        avg_blocks_last10.append(last_10_games["BLK"].mean())
        avg_turnovers_last10.append(last_10_games["TOV"].mean())
        avg_threepointers_last10.append(last_10_games["3P"].mean())
        
        
        
        
       

        
        

    # Assign new columns
    df["AvgFantPoints"] = avg_fantasy
    df["AvgFantPointsLast10"] = avg_fantasy_last10
    df["AvgMPTime"] = avg_mp_time
    df["AvgMPTimeLast10"] = avg_mp_time_last10
    df["AvgPoints"] = avg_points
    df["AvgPointsLast10"] = avg_points_last10
    df["AvgAssists"] = avg_assists
    df["AvgAssistsLast10"] = avg_assists_last10
    df["AvgRebounds"] = avg_rebounds
    df["AvgReboundsLast10"] = avg_rebounds_last10
    df["AvgSteals"] = avg_steals
    df["AvgStealsLast10"] = avg_steals_last10
    df["AvgBlocks"] = avg_blocks
    df["AvgBlocksLast10"] = avg_blocks_last10
    df["AvgTurnovers"] = avg_turnovers
    df["AvgTurnoversLast10"] = avg_turnovers_last10
    df["AvgThreePointers"] = avg_threepointers
    df["AvgThreePointersLast10"] = avg_threepointers_last10
    

    return df

In [14]:
def add_injury_counts(df):
    # Ensure correct data types
    
    
    # Create empty lists for new columns
    prev_inj_count = []
    opp_inj_count = []
    opp_prev_inj_count = []

    # Convert Date to datetime if not already
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

    # Create a lookup table for previous injuries by team
    prev_injury_lookup = {}

    # Iterate over each row
    for idx, row in tqdm(df.iterrows()):
        team = row["TeamAbv"]
        opp_team = row["OpponentTeamAbv"]
        game_id = row["GameID"]

        # Get previous injury count for the team
        prev_value = prev_injury_lookup.get(team, None)
        prev_inj_count.append(prev_value)

        # Get opponent's injury count for the current game
        opp_injury = df[(df["GameID"] == game_id) & (df["TeamAbv"] == opp_team)]["InjTeamateCount"]
        opp_inj_count.append(opp_injury.mean() if not opp_injury.empty else None)

        # Get previous injury count for the opponent
        opp_prev_value = prev_injury_lookup.get(opp_team, None)
        opp_prev_inj_count.append(opp_prev_value)

        # Update lookup table with the current team's injury count
        prev_injury_lookup[team] = row["InjTeamateCount"]

    # Assign new columns
    df["PrevInjuredCount"] = prev_inj_count
    df["OpponentInjTeamateCount"] = opp_inj_count
    df["OpponentPrevInjuredCount"] = opp_prev_inj_count

    return df
    

In [15]:
def cut_dataframe(nba_data, ending_date, team_name):
    # Filter data based on date and team criteria
    cut_data = nba_data[
        (nba_data["Date"] < ending_date) &
        (nba_data["Date"] > nba_data["Date"].min()) &
        (nba_data["OpponentTeamAbv"] == team_name) &
        (nba_data["Top7InTeam"] == True)
    ]

    # Group by position and calculate mean fantasy points
    cut_data = cut_data.groupby("Pos1", as_index=False).agg(AvgFantPoints=("FantasyPoints", "mean"))
    
    # Add team and date columns
    cut_data["team"] = team_name
    cut_data["date"] = ending_date

    # Compute team-level statistics
    team_games = nba_data[
        (nba_data["Date"] < ending_date) &
        (nba_data["Date"] >= nba_data["Date"].min()) &
        (nba_data["OpponentTeamAbv"] == team_name)
    ].groupby("GameID", as_index=False).agg(
        Loss=("WonGame", lambda x: max(x)),  # If any player won, the team lost
        Win=("WonGame", lambda x: max(x == False)),  # If any player lost, the team won
        PointDiff=("GamePointDiff", lambda x: -max(x))  # Reverse point diff
    )

    # Calculate win percentage and total point differential
    win_percentage = team_games["Win"].sum() / len(team_games) if len(team_games) > 0 else 0
    point_diff = team_games["PointDiff"].sum() if len(team_games) > 0 else 0

    # Add win percentage and point differential
    cut_data["WinPercentage"] = win_percentage
    cut_data["PointDiff"] = point_diff

    # Add overall average row
    overall_avg = pd.DataFrame([{
        "Pos1": "ALL",
        "AvgFantPoints": cut_data["AvgFantPoints"].mean(),
        "team": team_name,
        "date": ending_date,
        "WinPercentage": win_percentage,
        "PointDiff": point_diff
    }])

    # Append overall average row
    cut_data = pd.concat([cut_data, overall_avg], ignore_index=True)

    return cut_data


In [16]:
def whole_team_dataframe(nba_data, team_name):
    start_date = pd.to_datetime("2024-10-25")
    end_date = nba_data["Date"].max()
    date_sequence = pd.date_range(start=start_date, end=end_date, freq="D")

    # Loop through dates and combine results into one dataframe
    full_team_df = pd.concat([cut_dataframe(nba_data, date, team_name) for date in date_sequence], ignore_index=True)

    return full_team_df


In [17]:
def generate_full_nba_dataframe(nba_data):
    nba_team_abbreviations = [
        "ATL", "BOS", "BRK", "CHO", "CHI", "CLE", "DAL", "DEN", "DET", "GSW",
        "HOU", "IND", "LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NOP", "NYK",
        "OKC", "ORL", "PHI", "PHO", "POR", "SAC", "SAS", "TOR", "UTA", "WAS"
    ]

    # Loop through each team abbreviation and combine results into one dataframe
    full_nba_df = pd.concat([whole_team_dataframe(nba_data, team) for team in nba_team_abbreviations], ignore_index=True)

    # Rank within each date & position
    full_nba_df["FantPointsAgainstRank"] = full_nba_df.groupby(["date", "Pos1"])["AvgFantPoints"].rank(method="min")

    return full_nba_df

In [18]:
def add_opponent_info(MainDF):
    
    teams = generate_full_nba_dataframe(MainDF)
    
    print(MainDF)
    print(teams)
    print("Checkpoint1")


    MainDF['Date'] = pd.to_datetime(MainDF['Date'], format = 'mixed')
    teams['date'] = pd.to_datetime(teams['date'])

    teams_noposition =  teams.drop_duplicates(subset=['date', 'team'])
    nba_data_merged = MainDF.merge(
        teams_noposition[['date', 'team', 'FantPointsAgainstRank', 'PointDiff','WinPercentage']],  
        left_on=['Date', 'OpponentTeamAbv'], 
        right_on=['date', 'team'], 
        how='left'
    ).drop(columns=['team','date'])

    print(nba_data_merged)
    print("Checkpoint2")
    positional_ranks = teams.pivot(index=['date', 'team'], columns='Pos1', values='FantPointsAgainstRank')
    positional_ranks = positional_ranks.reset_index()

    nba_data_merged = nba_data_merged.merge(
        positional_ranks, 
        left_on=['Date', 'OpponentTeamAbv'], 
        right_on=['date', 'team'], 
        how='left'
    ).drop(columns=['team','date'])

    nba_data_merged.rename(columns = {'PointDiff':'OpponentPointDiff','WinPercentage':'OpponentWinPercentage','ALL':'OppFantDefense: ALL','C':'OppFantDefense: C','PF_y':'OppFantDefense: PF','PG':'OppFantDefense: PG','SF':'OppFantDefense: SF','SG':'OppFantDefense: SG', 'PF_x':'PF'}, inplace = True)

    
    print(nba_data_merged)
    print("Checkpoint3")
    
    
    return(nba_data_merged)

In [19]:
def get_boxscore_link(df):
    
    if int(df['Month#'].iloc[0]) < 10:
        month = '0' + str(df['Month#'].iloc[0])
    else:
        month = df['Month#'].iloc[0]
    year = df['Year'].iloc[0]
    day = df['Day'].iloc[0]
    hometeam = df['HomeTeamAbv'].iloc[0]
    
    link = f'https://www.basketball-reference.com/boxscores/{year}{month}{day}0{hometeam}.html'
    print(link)
    return link
    

In [38]:
def player_scores_game(GameDF):
    
    global positions
    
    columns_to_replace = [
    'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 
    'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-'
    ]
    URL = get_boxscore_link(GameDF)
    
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, features="html.parser")
    soup.prettify()
    tables = soup.find_all('table')
    
    
    try:
        inactive_section = soup.find('strong', string=lambda t: t and "Inactive:" in t)
        team_sections = inactive_section.find_next_siblings('span')
        team_inactives = {}  

        for team_section in team_sections:
            team_name = team_section.text.strip() 


            players = []
            sibling = team_section.find_next_sibling()
            while sibling and sibling.name == 'a':  
                players.append(sibling.text.strip())
                sibling = sibling.find_next_sibling()

            # Store the team and its players
            if players:
                team_inactives[team_name] = players
        rows = []
        for team, players in team_inactives.items():
            for player in players:
                row = {'Starters': player, 'Injured': True, 'TeamAbv': team}  
                rows.append(row)

        new_df = pd.DataFrame(rows)
        team_inactive_counts = {}
        team_inactive_counts = {team: len(players) for team, players in team_inactives.items()}
        
    except:
        new_df = pd.DataFrame()
        
    
    
    table_html1 = str(tables[0])
    df1 = pd.read_html(io.StringIO(table_html1))[0]
    df1.columns = df1.columns.droplevel(0)
    df1['Starting'] = [True if i < 5 else False for i in range(len(df1))]
    df1['Injured'] = False
    df1['TeamAbv'] = nba_team_abbreviations[GameDF['AwayTeam'].iloc[0]]
    name1 = df1['Starters'].iloc[0]
    
    df1_filtered = df1[~df1['Starters'].isin(['Team Totals', 'Reserves'])].copy()
    
    df1_filtered.loc[:, 'MP_sort'] = pd.to_numeric(df1_filtered['MP'], errors='coerce')
    top_non_startersdf1 = df1_filtered[df1_filtered['Starting'] == False].nlargest(2, 'MP_sort')
    
    df1['Top7InTeam'] = df1_filtered['Starting'] | df1_filtered.index.isin(top_non_startersdf1.index)
    
    
    
    
    GoodToGo = False
    i = 6
    
    while not GoodToGo:
        table_html2 = str(tables[i])
        df2 = pd.read_html(io.StringIO(table_html2))[0]
        #print(df2)
        #print("ABOVE IS THE SECOND DATAFRAME!")
        df2.columns = df2.columns.droplevel(0)
        df2['Starting'] = [True if i < 5 else False for i in range(len(df2))]
       
        df2['Injured'] = False
        df2['TeamAbv'] = nba_team_abbreviations[GameDF['HomeTeam'].iloc[0]]
        name2= df2['Starters'].iloc[0]
        #print(name2)
        #print(i)
        i = i + 1
        if name1 != name2:
            GoodToGo = True
        
    
    df2_filtered = df2[~df2['Starters'].isin(['Team Totals', 'Reserves'])].copy()
    
    df2_filtered.loc[:, 'MP_sort'] = pd.to_numeric(df2_filtered['MP'], errors='coerce')
    top_non_startersdf2 = df2_filtered[df2_filtered['Starting'] == False].nlargest(2, 'MP_sort')
    
    df2['Top7InTeam'] = df2_filtered['Starting'] | df2_filtered.index.isin(top_non_startersdf2.index)
    
    
    
    
#     for i in range(len(tables)):
#         print("                                    ")
#         print("                                    ")
#         print("                                    ")
#         print(i)
#         print("                                    ")
#         print("                                    ")
#         print("                                    ")
#         table_html0 = str(tables[i])
#         df0 = pd.read_html(io.StringIO(table_html0))[0]
#         print(df0)
    
    
    #Do a first run of didnotplay = injured, then do if did not play = false?
    # or do did not play be an or statement just the whole thing
    
    
    
    df_merged = pd.concat([df1, df2, new_df], ignore_index=True, sort=False)
    #df_merged.columns = df_merged.columns.droplevel(0)
    
    df_merged = df_merged[df_merged['Starters'] != 'Team Totals']
    df_merged = df_merged[df_merged['Starters'] != 'Reserves']

    
    df_merged['Date'] = GameDF['Date'].iloc[0]
    df_merged['Year'] = df_merged['Date'].dt.year
    df_merged['Day'] = df_merged['Date'].dt.day.astype(str).str.zfill(2)
    df_merged['Month#'] = df_merged['Date'].dt.month
    df_merged['MonthName'] = df_merged['Date'].dt.month_name().str.lower()
    df_merged['Day_of_week'] = df_merged['Date'].dt.day_name()
    df_merged['TeamName'] = df_merged['TeamAbv'].map(nba_team_names)
    df_merged['Home'] = df_merged['TeamName'].apply(lambda x: x == GameDF['HomeTeam'].iloc[0])
    df_merged['DidNotPlay'] = (df_merged['FG'] == df_merged['MP']) | df_merged['Injured']
    df_merged[df_merged['Injured']] = df_merged.fillna(0)
    
    
    for col_replace in columns_to_replace:
        df_merged.loc[df_merged['DidNotPlay'], col_replace] = 0
      
    
    df_merged['MPTimeDelta'] = df_merged['MP'].apply(
    lambda x: pd.to_timedelta("00:" + str(x)) if ':' in str(x) else pd.to_timedelta(0))
    df_merged['OpponentTeam'] = df_merged['Home'].apply(lambda x: GameDF['AwayTeam'].iloc[0] if x else GameDF['HomeTeam'].iloc[0])
    df_merged['OpponentTeamAbv'] = df_merged['Home'].apply(lambda x: GameDF['AwayTeamAbv'].iloc[0] if x else GameDF['HomeTeamAbv'].iloc[0])
    df_merged['WonGame'] = df_merged['Home'].apply(lambda x: GameDF['HomeWin'].iloc[0] if x else GameDF['AwayWin'].iloc[0])
    df_merged['GamePointDiff'] = df_merged['Home'].apply(lambda x: GameDF['HomePointDiff'].iloc[0] if x else GameDF['AwayPointDiff'].iloc[0])
    try:
        df_merged['Start(ET)'] = GameDF['Start (ET)'].iloc[0]
    except:
        df_merged['Start(ET)'] = np.NaN
    df_merged['Overtime'] = GameDF['Overtime'].iloc[0]
    df_merged['Attend.'] = GameDF['Attend.'].iloc[0]
    df_merged['InSeasonTournament'] = GameDF['InSeasonTournament'].iloc[0]
    
    
    
    df_merged['Starting'] = df_merged['Starting'].apply(lambda x: False if x == 0 else x)
    df_merged['PTS'] = pd.to_numeric(df_merged['PTS'], errors='coerce')
    df_merged['TRB'] = pd.to_numeric(df_merged['TRB'], errors='coerce')
    df_merged['AST'] = pd.to_numeric(df_merged['AST'], errors='coerce')
    df_merged['STL'] = pd.to_numeric(df_merged['STL'], errors='coerce')
    df_merged['BLK'] = pd.to_numeric(df_merged['BLK'], errors='coerce')
    df_merged['TOV'] = pd.to_numeric(df_merged['TOV'], errors='coerce')
    df_merged['3P'] = pd.to_numeric(df_merged['3P'], errors='coerce')
    df_merged['tens'] = ((df_merged['PTS'] >= 10) * 1) + ((df_merged['TRB'] >= 10) * 1) + ((df_merged['AST'] >= 10) * 1)+ ((df_merged['STL'] >= 10) * 1)+ ((df_merged['BLK'] >= 10) * 1)
    df_merged['FantasyPoints'] = (df_merged['PTS'] * .5) + (df_merged['TRB'] * 1) + (df_merged['AST'] * 1) + (df_merged['STL'] * 2) + (df_merged['BLK'] * 2) + (df_merged['TOV'] * -1) + (df_merged['3P'] * .5) + ((df_merged['PTS'] >= 40).astype(int) * 2) + ((df_merged['PTS'] >= 50).astype(int) * 2) + ((df_merged['tens'] >= 2).astype(int) * 1) + ((df_merged['tens'] >= 3).astype(int) * 2)
    df_merged['GameID'] = GameDF['GameID'].iloc[0]
    try:
        df_merged['InjTeamateCount'] = df_merged['TeamAbv'].map(team_inactive_counts)
    except:
        df_merged['InjTeamateCount'] = 0
    df_merged['Top7InTeam'] = df_merged['Top7InTeam'].apply(lambda x: False if x == 0 else x)
    
    df_merged = df_merged[['Date','Starters','TeamName','WonGame','Injured','DidNotPlay','FantasyPoints','MPTimeDelta','MP','FG','FGA','FG%','3P','3PA','3P%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','GmSc','+/-','TeamAbv','Year','Day','Month#','MonthName','Day_of_week','Home','GamePointDiff','Start(ET)','Overtime','Attend.','InSeasonTournament','GameID','OpponentTeam','OpponentTeamAbv','InjTeamateCount', 'Starting','Top7InTeam']]
    #df_merged = df_merged.rename(columns = {
    #    'Starters'
    #    
    #})
    
    
    return df_merged

#stats = player_scores_game(smalldfdf)

In [21]:
def get_game_dates(year, month):  
    session = requests.Session()
    #print(year, month)
    URL = f'https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html'
    #print(URL)
    response = session.get(URL)
    #print(response.status_code)
    soup = BeautifulSoup(response.text, features="html.parser")
    soup.prettify()
    tables = soup.find_all('table')
    #print(len(tables))
    table_html = str(tables[0])
    df = pd.read_html(io.StringIO(table_html))[0]
    df = df[~(df == 'Playoffs').all(axis=1)]
    df['PTS'] = pd.to_numeric(df['PTS'], errors='coerce').astype('Int64')
    df['PTS.1'] = pd.to_numeric(df['PTS.1'], errors='coerce').astype('Int64')
    df['Attend.'] = pd.to_numeric(df['Attend.'], errors='coerce').astype('Int64')
    
    #print(year,month)
    #print(df)
#     print(df['Date'])
#     print("BEFORE BEFORE BEFORE BEFORE ERROR") 
    df['Date'] = pd.to_datetime(df['Date'])
    #print("AFTER AFTER AFTER AFTER AFTER ERROR")
    df['Year'] = df['Date'].dt.year
    df['Day'] = df['Date'].dt.day.astype(str).str.zfill(2)
    df['Month#'] = df['Date'].dt.month
    df['MonthName'] = df['Date'].dt.month_name().str.lower()
    df['GameFinished'] = df['Date'] < (pd.Timestamp('today') - pd.Timedelta(days=1))
    
    if year > 2000:
        
        df = df.rename(columns = {
            'Visitor/Neutral':'AwayTeam',
            'PTS':'AwayPTS',
            'PTS.1':'HomePTS',
            'Home/Neutral' : 'HomeTeam',
            'Unnamed: 7' : 'Overtime',
            'Notes':'InSeasonTournament'
        })
        
        df = df.drop(['Unnamed: 6', 'LOG','Arena'], axis = 1)
    
    else:
        df = df.rename(columns = {
            'Visitor/Neutral':'AwayTeam',
            'PTS':'AwayPTS',
            'PTS.1':'HomePTS',
            'Home/Neutral' : 'HomeTeam',
            'Unnamed: 6' : 'Overtime',
            'Notes':'InSeasonTournament'
        })
        
        df = df.drop(['Unnamed: 5', 'LOG','Arena'], axis = 1)

    df['Overtime'] = df['Overtime'].apply(lambda x: True if x == 'OT' else False)
    df['HomeTeamAbv'] = df['HomeTeam'].map(nba_team_abbreviations)
    df['AwayTeamAbv'] = df['AwayTeam'].map(nba_team_abbreviations)
    df['HomeWin'] = df['HomePTS'] > df['AwayPTS']
    df['AwayWin'] = df['AwayPTS'] > df['HomePTS']
    df['HomePointDiff'] = df['HomePTS'] - df['AwayPTS']
    df['AwayPointDiff'] = df['AwayPTS'] - df['HomePTS']
    df['GameID'] = df['Year'].astype(str) + df['Day'].astype(str) + df['Month#'].astype(str) + df['HomeTeamAbv']

    df['InSeasonTournament'] = df['InSeasonTournament'].apply(lambda x: True if x == 'In-Season Tournament' else False)

    
    return df

In [22]:
messed_up = get_game_dates(2001,'november')


In [23]:
normal = get_game_dates(2000,'november')

In [24]:
def get_game_dates_months(year):  
    session = requests.Session()
    
    URL = f'https://www.basketball-reference.com/leagues/NBA_{year}_games.html'
    response = session.get(URL)
    response.status_code
    soup = BeautifulSoup(response.text, features="html.parser")

    filter_div = soup.find('div', {"class": "filter"})


    month_links = filter_div.find_all('a')


    months = [link.text.lower().replace(' ', '-') for link in month_links]
    
    return months


In [25]:
get_game_dates(2019, 'march')

,Date,Start (ET),AwayTeam,AwayPTS,HomeTeam,HomePTS,Overtime,Attend.,InSeasonTournament,Year,...,Month#,MonthName,GameFinished,HomeTeamAbv,AwayTeamAbv,HomeWin,AwayWin,HomePointDiff,AwayPointDiff,GameID
0,2019-03-01,7:30p,Chicago Bulls,168,Atlanta Hawks,161,False,15267,False,2019,...,3,march,True,ATL,CHI,False,True,-7,7,2019013ATL
1,2019-03-01,7:30p,Charlotte Hornets,123,Brooklyn Nets,112,False,15578,False,2019,...,3,march,True,BRK,CHO,False,True,-11,11,2019013BRK
2,2019-03-01,8:00p,Washington Wizards,96,Boston Celtics,107,False,18624,False,2019,...,3,march,True,BOS,WAS,True,False,11,-11,2019013BOS
3,2019-03-01,8:00p,Portland Trail Blazers,117,Toronto Raptors,119,False,19800,False,2019,...,3,march,True,TOR,POR,True,False,2,-2,2019013TOR
4,2019-03-01,9:00p,New Orleans Pelicans,130,Phoenix Suns,116,False,14123,False,2019,...,3,march,True,PHO,NOP,False,True,-14,14,2019013PHO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2019-03-31,6:00p,Los Angeles Lakers,130,New Orleans Pelicans,102,False,18562,False,2019,...,3,march,True,NOP,LAL,False,True,-28,28,2019313NOP
220,2019-03-31,7:00p,Sacramento Kings,113,San Antonio Spurs,106,False,18407,False,2019,...,3,march,True,SAS,SAC,False,True,-7,7,2019313SAS
221,2019-03-31,8:00p,Washington Wizards,95,Denver Nuggets,90,False,17356,False,2019,...,3,march,True,DEN,WAS,False,True,-5,5,2019313DEN
222,2019-03-31,8:30p,Charlotte Hornets,90,Golden State Warriors,137,False,19596,False,2019,...,3,march,True,GSW,CHO,True,False,47,-47,2019313GSW


In [26]:
get_game_dates_months(2019)

['october',
 'november',
 'december',
 'january',
 'february',
 'march',
 'april',
 'may',
 'june']

In [27]:
def all_games_season(year):
    months = get_game_dates_months(year)
    total_df = pd.DataFrame()
    all_dataframes = []
    for month in months:
        time.sleep(5)
        df = get_game_dates(year, month)
        all_dataframes.append(df)
    
    total_df = pd.concat(all_dataframes)
    
    return total_df

In [28]:
def final_df(year,existing_df,numaddrows, add_all):
    
    all_games_df = all_games_season(year)
    try:
        
        all_games_df = all_games_df[~all_games_df['GameID'].isin(existing_df['GameID'])]
    except:
        pass
    
    all_games_df = all_games_df[all_games_df['GameFinished'] == True]
    total_df = existing_df
    
    if add_all:
        for i in range(len(all_games_df)):
            game_df = player_scores_game(all_games_df.iloc[[i]])
            total_df = pd.concat([total_df, game_df], ignore_index=True, sort=False)
            time.sleep(4.15)
            print(f'Done with game #{i}')
            print(all_games_df.iloc[[i]])
    else:
        for i in range(numaddrows):
            game_df = player_scores_game(all_games_df.iloc[[i]])
            total_df = pd.concat([total_df, game_df], ignore_index=True, sort=False)
            time.sleep(4.15)
            print(f'Done with game #{i}')
            print(all_games_df.iloc[[i]])

    
    return total_df


#stats_2025 = final_df(2025)

In [29]:
def update_full_data_new(rows_to_add = 10, add_all = False, datayear = 2025):
    try: 
        existing_dataframe = pd.read_csv(f"../data/full_nba_data{datayear}.csv")
    except:
        existing_dataframe = pd.DataFrame()

    
        
    new_df = final_df(datayear, existing_dataframe, rows_to_add, add_all)
    new_df['Date'] = pd.to_datetime(new_df['Date'], format = 'mixed')

    new_df = new_df[['Date','Starters','TeamName','WonGame','Injured','DidNotPlay','FantasyPoints','MPTimeDelta','MP','FG','FGA','FG%','3P','3PA','3P%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','GmSc','+/-','TeamAbv','Year','Day','Month#','MonthName','Day_of_week','Home','GamePointDiff','Start(ET)','Overtime','Attend.','InSeasonTournament','GameID','OpponentTeam','OpponentTeamAbv','InjTeamateCount', 'Starting','Top7InTeam']]
    
    new_df.to_csv(f'../data/full_nba_data{datayear}.csv')
    return new_df
    

In [36]:
new_df_2 = update_full_data_new(datayear = 1996, rows_to_add = 1)
new_df_2

https://www.basketball-reference.com/boxscores/199511030ATL.html


IndexError: list index out of range

In [ ]:
new_df_2[new_df_2['GameID'] == '20060511SEA']

In [ ]:
def get_player_names(nba_data):
    unique_names = nba_data['Starters'].unique()
    return list(unique_names)

get_player_names(new_df_2)

In [ ]:
# for i in range(35):
#     new_df_2 = update_full_data_new(datayear = 2014, rows_to_add = 14, add_all = False)
#     print(f"RESET DONE WITH LOOP #{i+1}")
#     print("                            ")
#     print("//%/%?%?%?%/%?%?%?%?/%?///%%")
#     print("                            ")
#     print("//%/%?%?%?%/%?%?%?%?/%?///%%")
#     print("                            ")
#     print("//%/%?%?%?%/%?%?%?%?/%?///%%")
#     print("                            ")
#     print("//%/%?%?%?%/%?%?%?%?/%?///%%")
#     print("                            ")
#     print("//%/%?%?%?%/%?%?%?%?/%?///%%")

In [40]:
for j in [2000,1999,1998,1997,1996,1995]:
    
    
    
    for i in range(100):
        try:      
            new_df_2 = update_full_data_new(datayear = j, rows_to_add = 14, add_all = False)
        except:
            break

    
    new_df_2 = update_full_data_new(datayear = j, rows_to_add = 17, add_all = True)

https://www.basketball-reference.com/boxscores/199912250LAL.html
Done with game #0
          Date           AwayTeam  AwayPTS            HomeTeam  HomePTS  \
169 1999-12-25  San Antonio Spurs       93  Los Angeles Lakers       99   

     Overtime  Attend.  InSeasonTournament  Year Day  Month# MonthName  \
169     False    18997               False  1999  25      12  december   

     GameFinished HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  \
169          True         LAL         SAS     True    False              6   

     AwayPointDiff       GameID  
169             -6  19992512LAL  
https://www.basketball-reference.com/boxscores/199912260CLE.html
Done with game #1
          Date         AwayTeam  AwayPTS             HomeTeam  HomePTS  \
170 1999-12-26  Toronto Raptors       98  Cleveland Cavaliers       95   

     Overtime  Attend.  InSeasonTournament  Year Day  Month# MonthName  \
170     False    14135               False  1999  26      12  december   

     GameFi

https://www.basketball-reference.com/boxscores/199912270LAL.html
Done with game #0
          Date          AwayTeam  AwayPTS            HomeTeam  HomePTS  \
183 1999-12-27  Dallas Mavericks      106  Los Angeles Lakers      108   

     Overtime  Attend.  InSeasonTournament  Year Day  Month# MonthName  \
183     False    18997               False  1999  27      12  december   

     GameFinished HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  \
183          True         LAL         DAL     True    False              2   

     AwayPointDiff       GameID  
183             -2  19992712LAL  
https://www.basketball-reference.com/boxscores/199912270ORL.html
Done with game #1
          Date                AwayTeam  AwayPTS       HomeTeam  HomePTS  \
184 1999-12-27  Minnesota Timberwolves      107  Orlando Magic      105   

     Overtime  Attend.  InSeasonTournament  Year Day  Month# MonthName  \
184     False    14703               False  1999  27      12  december   

     GameFi

https://www.basketball-reference.com/boxscores/199912290GSW.html
Done with game #0
          Date            AwayTeam  AwayPTS               HomeTeam  HomePTS  \
197 1999-12-29  Philadelphia 76ers       97  Golden State Warriors       94   

     Overtime  Attend.  InSeasonTournament  Year Day  Month# MonthName  \
197     False    15346               False  1999  29      12  december   

     GameFinished HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  \
197          True         GSW         PHI    False     True             -3   

     AwayPointDiff       GameID  
197              3  19992912GSW  
https://www.basketball-reference.com/boxscores/199912290LAL.html
Done with game #1
          Date      AwayTeam  AwayPTS            HomeTeam  HomePTS  Overtime  \
198 1999-12-29  Phoenix Suns       87  Los Angeles Lakers      103     False   

     Attend.  InSeasonTournament  Year Day  Month# MonthName  GameFinished  \
198    18997               False  1999  29      12  december  

https://www.basketball-reference.com/boxscores/199912300VAN.html
Done with game #0
          Date           AwayTeam  AwayPTS             HomeTeam  HomePTS  \
211 1999-12-30  San Antonio Spurs       98  Vancouver Grizzlies       88   

     Overtime  Attend.  InSeasonTournament  Year Day  Month# MonthName  \
211     False    14403               False  1999  30      12  december   

     GameFinished HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  \
211          True         VAN         SAS    False     True            -10   

     AwayPointDiff       GameID  
211             10  19993012VAN  
https://www.basketball-reference.com/boxscores/199912300WAS.html
Done with game #1
          Date         AwayTeam  AwayPTS            HomeTeam  HomePTS  \
212 1999-12-30  New York Knicks       89  Washington Wizards       86   

     Overtime  Attend.  InSeasonTournament  Year Day  Month# MonthName  \
212     False    19053               False  1999  30      12  december   

     GameFi

https://www.basketball-reference.com/boxscores/200001040MIA.html
Done with game #0
         Date             AwayTeam  AwayPTS    HomeTeam  HomePTS  Overtime  \
12 2000-01-04  Vancouver Grizzlies       91  Miami Heat       87     False   

    Attend.  InSeasonTournament  Year Day  Month# MonthName  GameFinished  \
12    19035               False  2000  04       1   january          True   

   HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  AwayPointDiff  \
12         MIA         VAN    False     True             -4              4   

        GameID  
12  2000041MIA  
https://www.basketball-reference.com/boxscores/200001040MIL.html
Done with game #1
         Date       AwayTeam  AwayPTS         HomeTeam  HomePTS  Overtime  \
13 2000-01-04  Atlanta Hawks      113  Milwaukee Bucks      116     False   

    Attend.  InSeasonTournament  Year Day  Month# MonthName  GameFinished  \
13    12739               False  2000  04       1   january          True   

   HomeTeamAbv AwayTe

...

Done with game #7
         Date             AwayTeam  AwayPTS       HomeTeam  HomePTS  Overtime  \
26 1998-03-04  Seattle SuperSonics       83  Orlando Magic       96     False   

    Attend.  InSeasonTournament  Year Day  Month# MonthName  GameFinished  \
26    17248               False  1998  04       3     march          True   

   HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  AwayPointDiff  \
26         ORL         SEA     True    False             13            -13   

        GameID  
26  1998043ORL  
https://www.basketball-reference.com/boxscores/199803040PHI.html
Done with game #8
         Date         AwayTeam  AwayPTS            HomeTeam  HomePTS  \
27 1998-03-04  Milwaukee Bucks       80  Philadelphia 76ers       87   

    Overtime  Attend.  InSeasonTournament  Year Day  Month# MonthName  \
27     False    12768               False  1998  04       3     march   

    GameFinished HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  \
27          True     

Done with game #7
         Date   AwayTeam  AwayPTS         HomeTeam  HomePTS  Overtime  \
40 1998-03-06  Utah Jazz      122  New Jersey Nets      115     False   

    Attend.  InSeasonTournament  Year Day  Month# MonthName  GameFinished  \
40    20049               False  1998  06       3     march          True   

   HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  AwayPointDiff  \
40         NJN         UTA    False     True             -7              7   

        GameID  
40  1998063NJN  
https://www.basketball-reference.com/boxscores/199803060ORL.html
Done with game #8
         Date             AwayTeam  AwayPTS       HomeTeam  HomePTS  Overtime  \
41 1998-03-06  Cleveland Cavaliers       89  Orlando Magic       91      True   

    Attend.  InSeasonTournament  Year Day  Month# MonthName  GameFinished  \
41    17248               False  1998  06       3     march          True   

   HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  AwayPointDiff  \
41        

Done with game #7
         Date           AwayTeam  AwayPTS                HomeTeam  HomePTS  \
54 1998-03-08  San Antonio Spurs       82  Portland Trail Blazers       78   

    Overtime  Attend.  InSeasonTournament  Year Day  Month# MonthName  \
54     False    21116               False  1998  08       3     march   

    GameFinished HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  \
54          True         POR         SAS    False     True             -4   

    AwayPointDiff      GameID  
54              4  1998083POR  
https://www.basketball-reference.com/boxscores/199803080VAN.html
Done with game #8
         Date         AwayTeam  AwayPTS             HomeTeam  HomePTS  \
55 1998-03-08  Toronto Raptors      106  Vancouver Grizzlies      113   

    Overtime  Attend.  InSeasonTournament  Year Day  Month# MonthName  \
55     False    16098               False  1998  08       3     march   

    GameFinished HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  \
55   

Done with game #7
         Date       AwayTeam  AwayPTS        HomeTeam  HomePTS  Overtime  \
68 1998-03-11  Atlanta Hawks      110  Boston Celtics      105     False   

    Attend.  InSeasonTournament  Year Day  Month# MonthName  GameFinished  \
68    18624               False  1998  11       3     march          True   

   HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  AwayPointDiff  \
68         BOS         ATL    False     True             -5              5   

        GameID  
68  1998113BOS  
https://www.basketball-reference.com/boxscores/199803110CHH.html
Done with game #8
         Date         AwayTeam  AwayPTS           HomeTeam  HomePTS  Overtime  \
69 1998-03-11  New York Knicks       78  Charlotte Hornets       85     False   

    Attend.  InSeasonTournament  Year Day  Month# MonthName  GameFinished  \
69    23890               False  1998  11       3     march          True   

   HomeTeamAbv AwayTeamAbv  HomeWin  AwayWin  HomePointDiff  AwayPointDiff  \
69  

KeyboardInterrupt: 

In [ ]:
#testing = add_fantasy_averages_new_new(existing_dataframe)

In [ ]:
#testing

